<a href="https://colab.research.google.com/github/DajeanArcila/biblioteca_pandas/blob/main/ML_predi%C3%A7ao_dolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Modelo de ML para Calculo Do Dolar no Proximo mes(05/2024)



In [56]:
import pandas as pd


subimos las tablas de reservas internacionales(valos en billones) y la tabla de histiroco de cotacion del dolar


In [57]:
df_DadosHistoricos= pd.read_csv("/content/USD_BRL Dados Históricos atualizado.csv")
df_DadosReservasInter= pd.read_csv("/content/reservas internacionales billones.csv.csv", sep=";")
#identificamos que el separador es ";" en reserva internacional para que separe los datos de la fecha correctamente

In [58]:
df_DadosHistoricos.tail()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
2512,05.09.2014,"2,2405","2,2428","2,2508","2,2310",NaN,"-0,12%"
2513,04.09.2014,"2,2431","2,2372","2,2473","2,2253",NaN,"0,27%"
2514,03.09.2014,"2,2370","2,2427","2,2427","2,2257",NaN,"-0,29%"
2515,02.09.2014,"2,2435","2,2462","2,2573","2,2380",NaN,"-0,10%"
2516,01.09.2014,"2,2458","2,2367","2,2481","2,2302",NaN,"0,44%"


In [59]:
df_DadosReservasInter.tail()

,DateTime,Reservas internacionais
115,01/09/2014,"375,513"
116,01/08/2014,"379,157"
117,01/07/2014,"376,792"
118,01/06/2014,"373,516"
119,01/05/2014,"368,752"


In [60]:
df_DadosReservasInter.head()

,DateTime,Reservas internacionais
0,01/04/2024,"351,599"
1,01/03/2024,"355,008"
2,01/02/2024,"352,705"
3,01/01/2024,"355,066"
4,01/12/2023,"355,034"


In [61]:
df_DadosHistoricos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Data      2517 non-null   object
 1   Último    2517 non-null   object
 2   Abertura  2517 non-null   object
 3   Máxima    2517 non-null   object
 4   Mínima    2517 non-null   object
 5   Vol.      1032 non-null   object
 6   Var%      2517 non-null   object
dtypes: object(7)
memory usage: 137.8+ KB


In [62]:
df_DadosHistoricos = df_DadosHistoricos.drop('Vol.', axis=1)

In [63]:
df_DadosHistoricos.head()

,Data,Último,Abertura,Máxima,Mínima,Var%
0,30.04.2024,"5,1934","5,1190","5,1975","5,1133","1,46%"
1,29.04.2024,"5,1189","5,1170","5,1227","5,1020","0,07%"
2,26.04.2024,"5,1151","5,1615","5,1673","5,1085","-0,87%"
3,25.04.2024,"5,1602","5,1472","5,1935","5,1099","0,28%"
4,24.04.2024,"5,1457","5,1263","5,1720","5,1238","0,38%"


In [64]:
df_DadosReservasInter.head()

,DateTime,Reservas internacionais
0,01/04/2024,"351,599"
1,01/03/2024,"355,008"
2,01/02/2024,"352,705"
3,01/01/2024,"355,066"
4,01/12/2023,"355,034"


In [65]:
df_DadosHistoricos.head()

,Data,Último,Abertura,Máxima,Mínima,Var%
0,30.04.2024,"5,1934","5,1190","5,1975","5,1133","1,46%"
1,29.04.2024,"5,1189","5,1170","5,1227","5,1020","0,07%"
2,26.04.2024,"5,1151","5,1615","5,1673","5,1085","-0,87%"
3,25.04.2024,"5,1602","5,1472","5,1935","5,1099","0,28%"
4,24.04.2024,"5,1457","5,1263","5,1720","5,1238","0,38%"


convertir la columna de data para data


In [66]:
df_DadosHistoricos.describe()


,Data,Último,Abertura,Máxima,Mínima,Var%
count,2517,2517,2517,2517,2517,2517
unique,2517,2338,2344,2335,2346,498
top,30.04.2024,"3,9040","3,1265","3,2553","3,1075","0,00%"
freq,1,5,3,4,4,55


In [67]:
df_DadosReservasInter.describe()

,DateTime,Reservas internacionais
count,120,120
unique,120,120
top,01/04/2024,"351,599"
freq,1,1


In [69]:
df_DadosHistoricos['Data'] = pd.to_datetime(df_DadosHistoricos['Data']).dt.strftime('%d/%m/%Y')
df_DadosReservasInter['DateTime'] = pd.to_datetime(df_DadosReservasInter['DateTime']).dt.strftime('%d/%m/%Y')


<ipython-input-69-99862a97e125>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_DadosHistoricos['Data'] = pd.to_datetime(df_DadosHistoricos['Data']).dt.strftime('%d/%m/%Y')


logo depois de fazer todo o tratamento dos dados, convertemos eles para se concatenaren correctamente as datas diarias dos historicos do dolar junto con as reservas internacionales pegando o valor mensal registrado por 30 dias


In [70]:
def adaptar_cifra_mes(df_DadosReservasInter):
  """
  Adapta la cifra del primer día de cada mes en la tabla de reservas internacionales.

  Args:
    reservas_internacionales: Un DataFrame que contiene la tabla de reservas internacionales.

  Returns:
    Un nuevo DataFrame con la cifra del primer día de cada mes adaptada para todos los días del mes.
  """

  # Obtener el primer día de cada mes
  primeros_dias_mes = df_DadosReservasInter.groupby('DateTime')['DateTime'].min().reset_index()

  # Unir la tabla de primeros días de mes con la tabla de reservas internacionales
  reservas_con_primeros_dias_mes = df_DadosReservasInter.merge(primeros_dias_mes, on='DateTime', how='left')

  # Rellenar los valores faltantes de 'cifra' con la 'cifra' del primer día de cada mes
  reservas_con_primeros_dias_mes['cifra'] = reservas_con_primeros_dias_mes['cifra'].fillna(reservas_con_primeros_dias_mes['cifra_x'])

  # Eliminar la columna 'cifra_x'
  reservas_con_primeros_dias_mes.drop('cifra_x', axis=1, inplace=True)

  return reservas_con_primeros_dias_mes


In [77]:
def preparar_datos_ml(reservas_adaptadas, fecha_inicio, fecha_fin):
  """
  Prepara los datos para entrenar un modelo de machine learning.

  Args:
    reservas_adaptadas: Un DataFrame que contiene la tabla de reservas internacionales adaptada (debe reemplazar `reservas_adaptadas`).
    fecha_inicio: La fecha de inicio del período de predicción.
    fecha_fin: La fecha de finalización del período de predicción.

  Returns:
    Un conjunto de datos listo para entrenar un modelo de machine learning.
  """

  # Seleccionar las columnas relevantes
  columnas_relevantes = ['data', 'dia_semana', 'cifra']  # Revise si las columnas son las correctas para su caso.
  datos_ml = reservas_adaptadas[columnas_relevantes]

  # Filtrar los datos por fecha
  datos_ml = datos_ml[(datos_ml['data'] >= '01/04/2024') & (datos_ml['data'] <= '01/05/2014')]

  # Convertir la columna 'dia_semana' en variables categóricas
  datos_ml['dia_semana'] = pd.get_dummies(datos_ml['dia_semana'], drop_first=True)

  # Separar las características y la variable objetivo
  X = datos_ml.drop('cifra', axis=1)
  y = datos_ml['cifra']

  return X, y


In [81]:
df_DadosHistoricos.head()

,Data,Último,Abertura,Máxima,Mínima,Var%
0,30/04/2024,"5,1934","5,1190","5,1975","5,1133","1,46%"
1,29/04/2024,"5,1189","5,1170","5,1227","5,1020","0,07%"
2,26/04/2024,"5,1151","5,1615","5,1673","5,1085","-0,87%"
3,25/04/2024,"5,1602","5,1472","5,1935","5,1099","0,28%"
4,24/04/2024,"5,1457","5,1263","5,1720","5,1238","0,38%"


In [1]:
df_DadosReservasInter.head()

NameError: name 'df_DadosReservasInter' is not defined